In [1]:
from PIL import Image
import numpy as np
import os
import pickle
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torch.utils.data import Dataset, DataLoader

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
# import umap.umap_ as umap
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

from skimage.transform import resize
from torchvision import transforms

import torch
import torch.nn as nn

import time


d:\zsh\graduation\grad_env_take_2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data requirements
* we need: image name or id (to distinguish link back to the main df),simple_tsne, meaningful_tsne, pca,bottleneck_activations_embedding, IoU values, image path, prediction path
* existing information: name, dataset, label_path, class ratios,
* not useful for continuous domains: similar_image_paths

Sept 20:
Need to add the umap embedding for both input and model activations

#  Load images and the labels (**Cityscapes**)

In [2]:
# path of the image folders for the model
image_path_cityscapes = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\original_cityscapes_inputs" # 500 images in total
label_path_cityscapes = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\cityscapes_labels_sample"
# path of image folders for displaying in the system

start = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image"
relative_img_path_cityscapes = os.path.relpath(image_path_cityscapes, start)
relative_label_path_cityscapes = os.path.relpath(label_path_cityscapes, start)

In [3]:
# read in the relavant data
pickle_file = os.path.join("D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\\5_classes_preprocessed", "validation_images.pkl")
with open(pickle_file, 'rb') as f:
    X_test = pickle.load(f)

pickle_file = os.path.join("D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\\5_classes_preprocessed", "validation_label_classes.pkl")
with open(pickle_file, 'rb') as f:
    Y_test = pickle.load(f)

# load image and labels from pickle file
X_test = np.array(X_test)
Y_test = np.array(Y_test)

# subset of the image
sample_number=200

# # get the sample images and labels
# subset_images = X_test[:sample_number]
cityscapes_labels_sample = Y_test[:sample_number]
# no subset images

# get the names
# cityscapes_names= os.listdir(image_path_cityscapes)
cityscapes_names_sample=[]
for idx in range(sample_number):
    name = str(idx)+".jpeg"
    cityscapes_names_sample.append(name)

In [4]:
print(cityscapes_names_sample[:10])

['0.jpeg', '1.jpeg', '2.jpeg', '3.jpeg', '4.jpeg', '5.jpeg', '6.jpeg', '7.jpeg', '8.jpeg', '9.jpeg']


In [5]:
# Generate some initial info based on the names: name, image_path, and also dataset
cityscapes_initial_info = [] # include name, image path and dataset name
cityscapes_img_sample = []

for i,name in enumerate(cityscapes_names_sample):
    image = Image.open(image_path_cityscapes+"\\"+name).convert("RGB")
    image_path = relative_img_path_cityscapes+"\\"+name
    label_path = relative_label_path_cityscapes+"\\"+name
    
    cityscapes_img_sample.append(np.array(image))
    cityscapes_initial_info.append((name,image_path,label_path))

cityscapes_initial_info = np.array(cityscapes_initial_info)

In [6]:
initial_info_df = pd.DataFrame(cityscapes_initial_info,columns = ["name", "image_path","label_path"]) # no noise images yet
initial_info_df.head()

,name,image_path,label_path
0,0.jpeg,dataset\original_cityscapes_inputs\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg
1,1.jpeg,dataset\original_cityscapes_inputs\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg
2,2.jpeg,dataset\original_cityscapes_inputs\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg
3,3.jpeg,dataset\original_cityscapes_inputs\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg
4,4.jpeg,dataset\original_cityscapes_inputs\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg


# Class ratio for the images

In [7]:
def get_class_distribution(labels):
    class_dist = []
    for label in labels:
        class_for_label = []
        for i in range(6):
            element_count = np.count_nonzero(label==i)
            class_for_label.append(element_count/(256*256))
        class_dist.append(class_for_label)
#         class_for_label= class_for_label# change to another size if the image size is changed
    class_dist = np.array(class_dist)
    return class_dist

In [8]:
class_distribution= get_class_distribution(cityscapes_labels_sample)
class_df = pd.DataFrame(class_distribution, columns =["others","road","sidewalk","vegetation","sky","car"]) 
class_df.head()

,others,road,sidewalk,vegetation,sky,car
0,0.372009,0.320770,0.100510,0.183990,0.020248,0.002472
1,0.539703,0.414398,0.041168,0.000565,0.004166,0.000000
2,0.462753,0.354187,0.029633,0.121323,0.018356,0.013748
3,0.425674,0.381119,0.065292,0.101120,0.016296,0.010498
4,0.400284,0.329453,0.090469,0.174393,0.001465,0.003937


In [9]:
# this data frame is the "original" data before adding new noisy images
original_df = pd.merge(initial_info_df, class_df, left_index=True, right_index=True)

In [10]:
original_df

,name,image_path,label_path,others,road,sidewalk,vegetation,sky,car
0,0.jpeg,dataset\original_cityscapes_inputs\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,0.002472
1,1.jpeg,dataset\original_cityscapes_inputs\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,0.000000
2,2.jpeg,dataset\original_cityscapes_inputs\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,0.013748
3,3.jpeg,dataset\original_cityscapes_inputs\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,0.010498
4,4.jpeg,dataset\original_cityscapes_inputs\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,0.003937
...,...,...,...,...,...,...,...,...,...
195,195.jpeg,dataset\original_cityscapes_inputs\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,0.003189
196,196.jpeg,dataset\original_cityscapes_inputs\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,0.002884
197,197.jpeg,dataset\original_cityscapes_inputs\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,0.003357
198,198.jpeg,dataset\original_cityscapes_inputs\198.jpeg,dataset\cityscapes_labels_sample\198.jpeg,0.708481,0.000000,0.000000,0.236801,0.000381,0.054337


# Create new images based on the sample images

In [11]:
# noise_levels = [0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8]
noise_levels = [0,1,2,3,4,5]

A function to transform original data to noisy data

In [12]:
def transform_dataset(original_dataset,noise_level=0):
    new_dataset = []
    if noise_level == 0:
        return original_dataset
    for image in original_dataset:
        new_image = image+(noise_level*np.random.normal(0, (image.max() - image.min())/6., image.shape)) # (mean, sigma, image_shape)
        # sometimes the random results are out of the pixel range, portraying very weird values
        new_image = np.clip(new_image,0,255).astype("uint8")
        new_dataset.append(new_image)
    new_dataset = np.array(new_dataset)
    return new_dataset

In [13]:
# generate all the images for different noise levels
input_image_list = [transform_dataset(cityscapes_img_sample,noise_levels[i]) for i in range(len(noise_levels))]
input_image_array = np.array(input_image_list)
input_image_array.shape

(6, 200, 256, 256, 3)

Understand the path_list function

# Create df with all (noisy) images

In [14]:
noisy_image_folder = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\\noise_data\images"

In [15]:
path_list = [] # including the label path, 
for i,noisy_images in enumerate(input_image_array):
    noise_level = noise_levels[i]
    noise_level_folder = os.path.join(noisy_image_folder,str(noise_level))
    if not os.path.exists(noise_level_folder):
        os.makedirs(noise_level_folder)
    for j,image in enumerate(noisy_images):
        # save the noisy image
        img_object = Image.fromarray(image.astype(np.uint8))
        name = str(j)+".jpeg"
        # get the path of saved noisy image
        img_path = os.path.join(noise_level_folder,name)
        img_path_rel = os.path.relpath(img_path, start)
        #         uncomment this when generating noise images
        img_object.save(img_path)
        # also get the name and label path
#         original_row = original_df.loc[original_df["name"]==name]
#         label_path = original_row["label_path"].values[0]
#         instance=[name,img_path_rel,label_path]
#         for column in ratio_columns:
#             instance.append(original_row[column].values[0])        
        path_list.append(img_path_rel)

In [16]:
path_list[:5]

['dataset\\noise_data\\images\\0\\0.jpeg',
 'dataset\\noise_data\\images\\0\\1.jpeg',
 'dataset\\noise_data\\images\\0\\2.jpeg',
 'dataset\\noise_data\\images\\0\\3.jpeg',
 'dataset\\noise_data\\images\\0\\4.jpeg']

Get duplicated version of original df

In [17]:
# list of df duplicates with the corresponding noise level
repeat_count = 6
dfs = []
for i in range(repeat_count):
    df_temp = original_df.copy()
    df_temp['noise_level'] = noise_levels[i]
    dfs.append(df_temp)

In [18]:
# merge the list of dfs
noisy_df = pd.concat(dfs, ignore_index=True)

Replace the image paths, because only that column is changed for different noise levels

In [19]:
noisy_df["image_path"] = path_list
noisy_df

,name,image_path,label_path,others,road,sidewalk,vegetation,sky,car,noise_level
0,0.jpeg,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,0.002472,0
1,1.jpeg,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,0.000000,0
2,2.jpeg,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,0.013748,0
3,3.jpeg,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,0.010498,0
4,4.jpeg,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,0.003937,0
...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,0.003189,5
1196,196.jpeg,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,0.002884,5
1197,197.jpeg,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,0.003357,5
1198,198.jpeg,dataset\noise_data\images\5\198.jpeg,dataset\cityscapes_labels_sample\198.jpeg,0.708481,0.000000,0.000000,0.236801,0.000381,0.054337,5


# Embedding of inputs

In [20]:
combined_image_array = np.vstack(input_image_array)

In [21]:
combined_image_array.shape

(1200, 256, 256, 3)

In [22]:
# simple_combined_embedding =np.reshape(input_image_array,(len(combined_image_array)*sample_number, 256*256*3))
simple_combined_embedding = combined_image_array.reshape(combined_image_array.shape[0],-1)

# pca_50 = PCA(n_components=50)
# pca_embedding = pca_50.fit_transform(simple_combined_embedding)
# print(np.sum(pca_50.explained_variance_ratio_))
# tsne = TSNE(perplexity = 5)

tsne = TSNE()
simple_tsne_embedding = tsne.fit_transform(simple_combined_embedding)

In [23]:
pca = PCA(n_components=2)
pca_embedding = pca.fit_transform(simple_combined_embedding)
print(np.sum(pca.explained_variance_ratio_))

0.05194494515724889


In [24]:
reducer = umap.UMAP()
input_umap_embedding = reducer.fit_transform(simple_combined_embedding)

In [25]:
print(input_umap_embedding.shape)

(1200, 2)


**Pre-trained classifier**

In [26]:
cd D:\zsh\graduation\ViTs-vs-CNNs

D:\zsh\graduation\ViTs-vs-CNNs


In [27]:
import torch
import torch.nn as nn
import global_val

from models.ghost_bn import GhostBN2D_ADV
from models.advresnet_gbn_gelu import Affine
import models.advresnet_gbn_gelu as advres
from main_adv_res import EightBN

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classifier_model = advres.__dict__["resnet50"](norm_layer = EightBN)
weights_path = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\models\\advres50_gelu.pth"
weight_dict = torch.load(weights_path,map_location=device)["model"]

classifier_model.load_state_dict(weight_dict)

<All keys matched successfully>

In [28]:
cd D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\jupyter

D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\jupyter


(added second_part_embedding back after the last run)

In [29]:
def get_representation_for_image(image,label):
    # resize the image and transform it to tensor # [256,256,3] -> [224,224,3] 
    resized_image = resize(image, (224, 224,3))
    resized_image_tensor = torch.from_numpy(resized_image)
    resized_image_tensor = torch.permute(resized_image_tensor, (2, 0, 1))
    resized_image_tensor = resized_image_tensor[None,:] # [3,224,224] -> [1,3,224,224] (because model takes 4D input)
    
    # dictionary for the activations
    activations = {}

    def get_activations(name):
        def hook(model, input, output):
            activations[name] = output.detach()
        return hook

    # use the output of (avgpool) layer because it is the same as the input of last layer
    h = classifier_model.avgpool.register_forward_hook(get_activations("input_last_layer"))

    classifier_model.eval()
    classifier_model.sing=True
    classifier_model.training=False
    out = classifier_model(resized_image_tensor.float(),label)
    
    # remove the hook
    h.remove()

    first_part_embedding = torch.flatten(activations["input_last_layer"][0])
    # the second part of the embedding is size of each class
    #  (this section of code is overlapping with generating class distribution, improve this for a better result)   
    second_part_embedding=[]
    for i in range(6):
        class_size = np.count_nonzero(label == i)
        second_part_embedding.append(class_size)
    embedding = np.concatenate((first_part_embedding.numpy(),np.array(second_part_embedding)),axis = 0)
    
    return embedding

In [31]:
print(combined_image_array.shape)

(1200, 256, 256, 3)


In [32]:
embedding_list = []
for i, image in enumerate(combined_image_array): # i is not used, could remove the enumerate
    embedding = get_representation_for_image(image,"__") # y_label is never used, so just use replacement
    embedding_list.append(embedding)

embedding_arr = np.array(embedding_list)

# only use t-SNE for the classifier embedding
# tsne = TSNE(perplexity = 10) (didn't change much)
tsne = TSNE()
meaningful_tsne_embedding = tsne.fit_transform(embedding_arr)

In [34]:
# df_full = pd.read_csv("D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\\noisy_data_sample_200.csv",index_col="id")

In [36]:
# df_full.columns

Index(['name', 'noise_level', 'image_path', 'label_path', 'prediction_path',
       'other_ratio', 'road_ratio', 'sidewalk_ratio', 'vegetation_ratio',
       'sky_ratio', 'car_ratio', 'simple_tsne_1', 'simple_tsne_2',
       'meaningful_tsne_1', 'meaningful_tsne_2', 'pca_1', 'pca_2', 'umap_1',
       'umap_2', 'bottleneck_pca_embedding', 'bottleneck_tsne_embedding',
       'overall_iou', 'other_iou', 'road_iou', 'sidewalk_iou',
       'vegetation_iou', 'sky_iou', 'car_iou'],
      dtype='object')

In [38]:
# df_full[["meaningful_tsne_1","meaningful_tsne_2"]] = meaningful_tsne_embedding

In [39]:
# df_full

,name,noise_level,image_path,label_path,prediction_path,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,...,umap_2,bottleneck_pca_embedding,bottleneck_tsne_embedding,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou
id,,,,,,,,,,,,,,,,,,,,,
0,0.jpeg,0,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,dataset\noise_data\predictions\0\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,...,2.906299,"[[-0.013326136395335197, -0.30401840806007385]...","[[-8.655488967895508, 15.489809036254883], [-1...",0.823029,0.665488,0.949372,0.522178,0.559989,0.869651,0.066764
1,1.jpeg,0,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,dataset\noise_data\predictions\0\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,...,1.605131,"[[0.1603696197271347, -0.22683094441890717], [...","[[-14.934941291809082, 11.525232315063477], [-...",0.694122,0.598210,0.741562,0.246331,0.001455,0.480565,0.000000
2,2.jpeg,0,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,dataset\noise_data\predictions\0\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,...,5.091578,"[[0.019657980650663376, -0.3139321804046631], ...","[[-1.8404988050460815, 13.750861167907715], [-...",0.904343,0.809380,0.970170,0.646972,0.827567,0.941647,0.194177
3,3.jpeg,0,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,dataset\noise_data\predictions\0\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,...,4.119893,"[[0.1766330897808075, -0.19020713865756989], [...","[[-16.590892791748047, 5.171955108642578], [-0...",0.915451,0.850384,0.945627,0.707880,0.766526,0.884749,0.261182
4,4.jpeg,0,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,dataset\noise_data\predictions\0\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,...,3.517825,"[[0.924606204032898, 0.10404495894908905], [1....","[[-13.471735000610352, 9.674683570861816], [-1...",0.901062,0.769480,0.971295,0.837572,0.880520,0.296774,0.063342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,5,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,dataset\noise_data\predictions\5\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,...,4.354430,"[[0.520260214805603, -0.023931534960865974], [...","[[-12.483431816101074, -0.7725005149841309], [...",0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841
1196,196.jpeg,5,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,dataset\noise_data\predictions\5\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,...,2.663829,"[[0.3765714466571808, -0.041820090264081955], ...","[[-4.8248372077941895, -16.330381393432617], [...",0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000
1197,197.jpeg,5,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,dataset\noise_data\predictions\5\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,...,2.381734,"[[0.10845088958740234, 0.3767417371273041], [0...","[[-17.50540542602539, 6.707708358764648], [-7....",0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000


In [40]:
# df_full.to_csv("noisy_data_sample_200_meaningful_tsne.csv",index=True)

In [33]:
# meaningful_tsne_embedding.shape

(1200, 2)

In [34]:
full_embedding = np.concatenate((simple_tsne_embedding,meaningful_tsne_embedding,pca_embedding,input_umap_embedding),axis=1)
df_input_embedding = pd.DataFrame(full_embedding, columns=["simple_tsne_1","simple_tsne_2","meaningful_tsne_1","meaningful_tsne_2","pca_1","pca_2","umap_1","umap_2"])

In [35]:
noisy_df = pd.concat((noisy_df,df_input_embedding),axis = 1)

In [36]:
noisy_df # should contains 28 columns

,name,image_path,label_path,others,road,sidewalk,vegetation,sky,car,noise_level,simple_tsne_1,simple_tsne_2,meaningful_tsne_1,meaningful_tsne_2,pca_1,pca_2,umap_1,umap_2
0,0.jpeg,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,0.002472,0,22.514233,11.008042,-8.188470,-58.667126,-8000.299948,9617.183118,-0.698437,2.906299
1,1.jpeg,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,0.000000,0,-16.893806,0.480573,2.119193,-49.834877,12156.509965,-373.437321,3.069255,1.605131
2,2.jpeg,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,0.013748,0,5.917877,-10.341024,-1.836681,-65.091690,7763.920062,8242.034722,2.172739,5.091578
3,3.jpeg,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,0.010498,0,-8.857928,20.702034,-4.180655,-54.550453,4467.366231,1137.464870,3.008074,4.119893
4,4.jpeg,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,0.003937,0,14.474420,6.152795,2.736611,-68.335037,-3133.875666,12602.637921,-0.037946,3.517825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,0.003189,5,29.113890,-20.679844,14.696610,46.164577,-8185.635411,-1282.660143,0.165843,4.354430
1196,196.jpeg,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,0.002884,5,-36.991146,-16.982988,12.274261,60.767994,-2523.660095,-9698.227061,3.703897,2.663829
1197,197.jpeg,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,0.003357,5,-29.894413,-9.286451,2.938514,57.684685,-3221.224951,-9677.253907,3.205728,2.381734
1198,198.jpeg,dataset\noise_data\images\5\198.jpeg,dataset\cityscapes_labels_sample\198.jpeg,0.708481,0.000000,0.000000,0.236801,0.000381,0.054337,5,33.440460,7.517469,17.635401,44.290855,-6195.172767,-1289.474219,1.021540,3.467603


# Dataloader and model

In [37]:
# class for loading data to dataloader
class CityscapesDataset(Dataset):
    
    def __init__(self, images, labels, noise_level = 0):
        self.images = images
        self.labels = labels
        self.noise_level = noise_level
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        if self.noise_level!=0:
            image = image+(self.noise_level*np.random.normal(0, (image.max() - image.min()), image.shape)).astype("uint8") # (mean, sigma, image_shape)
        image = self.transform(image)
        label = torch.from_numpy(label).long()
        return image, label
        
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) # normalize to control the "dynamic range" of activations of different layers
        ])
        return transform_ops(image)

In [38]:
# model structure
class UNet(nn.Module):
    
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.num_classes = num_classes
        self.contracting_11 = self.conv_block(in_channels=3, out_channels=64)
        self.contracting_12 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_21 = self.conv_block(in_channels=64, out_channels=128)
        self.contracting_22 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_31 = self.conv_block(in_channels=128, out_channels=256)
        self.contracting_32 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_41 = self.conv_block(in_channels=256, out_channels=512)
        self.contracting_42 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
        self.expansive_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_12 = self.conv_block(in_channels=1024, out_channels=512)
        self.expansive_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_22 = self.conv_block(in_channels=512, out_channels=256)
        self.expansive_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_32 = self.conv_block(in_channels=256, out_channels=128)
        self.expansive_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_42 = self.conv_block(in_channels=128, out_channels=64)
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=3, stride=1, padding=1)
        
    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels),
                                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels))
        return block
    
    def forward(self, X):
        contracting_11_out = self.contracting_11(X) # [-1, 64, 256, 256]
        contracting_12_out = self.contracting_12(contracting_11_out) # [-1, 64, 128, 128]
        contracting_21_out = self.contracting_21(contracting_12_out) # [-1, 128, 128, 128]
        contracting_22_out = self.contracting_22(contracting_21_out) # [-1, 128, 64, 64]
        contracting_31_out = self.contracting_31(contracting_22_out) # [-1, 256, 64, 64]
        contracting_32_out = self.contracting_32(contracting_31_out) # [-1, 256, 32, 32]
        contracting_41_out = self.contracting_41(contracting_32_out) # [-1, 512, 32, 32]
        contracting_42_out = self.contracting_42(contracting_41_out) # [-1, 512, 16, 16]
        middle_out = self.middle(contracting_42_out) # [-1, 1024, 16, 16]
        expansive_11_out = self.expansive_11(middle_out) # [-1, 512, 32, 32]
        expansive_12_out = self.expansive_12(torch.cat((expansive_11_out, contracting_41_out), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        expansive_21_out = self.expansive_21(expansive_12_out) # [-1, 256, 64, 64]
        expansive_22_out = self.expansive_22(torch.cat((expansive_21_out, contracting_31_out), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        expansive_31_out = self.expansive_31(expansive_22_out) # [-1, 128, 128, 128]
        expansive_32_out = self.expansive_32(torch.cat((expansive_31_out, contracting_21_out), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        expansive_41_out = self.expansive_41(expansive_32_out) # [-1, 64, 256, 256]
        expansive_42_out = self.expansive_42(torch.cat((expansive_41_out, contracting_11_out), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        output_out = self.output(expansive_42_out) # [-1, num_classes, 256, 256]
        return output_out,middle_out

In [39]:
model_path = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\models\\5-classes-U-Net-2023-03-09.pth"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model_ = UNet(num_classes=6).to(device)
model_.load_state_dict(torch.load(model_path,map_location=device))

cpu


<All keys matched successfully>

In [40]:
def IoU_per_class(prediction,label):
    prediction = prediction.detach().numpy()
    label = label.detach().numpy()
    # Loop over each class
    iou_list = []
    # Flatten label and class arrays
    flat_prediction = prediction.flatten()
    flat_label = label.flatten()

    for i in range(6):
        
        # Calculate intersection and union
        intersection = np.sum((flat_prediction == i) & (flat_label == i))
        union = np.sum((flat_prediction == i) | (flat_label == i))

        # Calculate IoU
        iou = intersection / (union + 1e-12)

        # Store IoU value in dictionary
        iou_list.append(iou)
    
    # calculate overall IoU
    intersection = len(np.where(flat_prediction == flat_label)[0])
    union = len(flat_label)
    overall_iou = intersection / (union + 1e-12)
    
    return overall_iou,iou_list

In [41]:
combined_image_array.shape

(1200, 256, 256, 3)

In [48]:
combined_labels = []

for _ in range(len(noise_levels)):
    for row in cityscapes_labels_sample:
        combined_labels.append(row)

combined_labels = np.array(combined_labels)

In [44]:
combined_labels.shape

(1200, 256, 256)

In [50]:
print(np.all(combined_labels[0,:,:]==combined_labels[200,:,:]))
# print(combined_labels[56,:,:])

True


In [51]:
# get labels for dataloader from the dataframe
# combined_labels = []
# combined_labels = np.tile(subset_labels,(len(noise_levels),1,1)) # repeat the array 4 times along the first axis, while keeping the other axes unchanged
# print(combined_labels.shape)

# construct data loader
batch_size = 1
noise_dataset = CityscapesDataset(combined_image_array, combined_labels,noise_level=0)
noise_loader = DataLoader(noise_dataset, batch_size=batch_size)

In [52]:
print(combined_image_array.shape)
print(combined_labels.shape)

(1200, 256, 256, 3)
(1200, 256, 256)


In [53]:
# class to color: for viewing labels
colors = [[  0,   0,   0],
          [128, 64, 128],# road
          [244, 35, 232], # sidewalk
          [107, 142, 35],# vegetation
          [70, 130, 180], # sky
          [0, 0, 142], # car
         ]

def class_to_color(labels):
    label_colors = np.zeros((256,256,3))
    
    for i,row in enumerate(labels):
        for j,pixel in enumerate(row):
            label_colors[i,j] = colors[pixel]
    
    return label_colors.astype(int)  # make each pixel value an integer to visualize it better

In [54]:
# get the model activations and iou for a dataset
def get_activations_and_iou(data_loader):
    activations_list = []
    overall_iou_list = []
    iou_by_class_list =[]
    predictions_list = []

    for image,label in tqdm(data_loader):
        # get prediction and bottleneck activations
        output,bottleneck_activations = model_(image)
        # transform the predictions to the corresponding classes
        output_class=torch.argmax(output, dim=1)
        # from 4d [1,1024,16,16] to 2d: [256,1024]
        instance_activation_reshaped = bottleneck_activations.clone().detach().numpy().squeeze(0).reshape(1024,256).transpose()
        activations_list.append(instance_activation_reshaped)
        
        overall_iou,iou_list = IoU_per_class(output_class,label)
        overall_iou_list.append(overall_iou)
        iou_by_class_list.append(iou_list)
        output_class = output_class.detach().numpy()[0]
        predictions_list.append(output_class)
    
    return np.array(activations_list),np.array(overall_iou_list),np.array(iou_by_class_list),np.array(predictions_list)

In [55]:
model_activations, overall_ious, ious_by_class, predictions = get_activations_and_iou(noise_loader)

100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [16:12<00:00,  1.23it/s]


In [56]:
print(predictions.shape)

(1200, 256, 256)


Save predictions to paths

In [57]:
# full and relative
noisy_prediction_folder = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\dataset\\noise_data\predictions"

In [58]:
# save also the relative paths of predictions, so that we could access it from the front-end
# (could make this a function)
prediction_paths = []
for i, prediction in enumerate(predictions):
    # name and noise level of the current image
    name = noisy_df.iloc[i]["name"]
    instance_noise_level = noisy_df.iloc[i]["noise_level"]
    # get the version of predictions with color channels
    prediction_img = class_to_color(prediction)
    
    # make a full path for the predictions
    full_folder_path = os.path.join(noisy_prediction_folder,str(instance_noise_level))
    prediction_path_full = os.path.join(full_folder_path,name)
    prediction_path_rel = os.path.relpath(prediction_path_full, start)
    
    # prediction images
    prediction_img = Image.fromarray(prediction_img.astype(np.uint8))
    prediction_img.save(prediction_path_full)
    prediction_paths.append(prediction_path_rel)

In [59]:
prediction_paths[:5]

['dataset\\noise_data\\predictions\\0\\0.jpeg',
 'dataset\\noise_data\\predictions\\0\\1.jpeg',
 'dataset\\noise_data\\predictions\\0\\2.jpeg',
 'dataset\\noise_data\\predictions\\0\\3.jpeg',
 'dataset\\noise_data\\predictions\\0\\4.jpeg']

# Create embedding for activations

In [60]:
model_activations_2d = model_activations.reshape(model_activations.shape[0]*model_activations.shape[1],-1) # 256 is 16x16, the activation "image"

In [61]:
model_activations_2d.shape

(307200, 1024)

In [62]:
scaler = MinMaxScaler()
scaler.fit(model_activations_2d)
activations_2d_scaled = scaler.transform(model_activations_2d)

In [3]:
# pickle_file = "activations_2d.pkl"
# with open(pickle_file, 'rb') as f:
#     activations_2d = pickle.load(f)

generate embedding with pca, umap and t-sne

In [64]:
with open('noise_activations_scaled_200_samples_9.22.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(activations_2d_scaled, file)

In [65]:
# Start the timer
start_time = time.time()
activations_pca_embedding = PCA(n_components=2).fit_transform(activations_2d_scaled)

# End the timer
end_time = time.time()

# Calculate and print the time taken
elapsed_time = end_time - start_time
print(f"Time taken for PCA: {elapsed_time:.4f} seconds")

KeyboardInterrupt: 

In [133]:
activations_2d_scaled.shape

(76800, 1024)

In [134]:
print(np.isnan(activations_2d_scaled).sum())
print(np.isinf(activations_2d_scaled).sum())
# no data corruption

0
0


In [137]:
# !pip install -U umap-learn

In [139]:
# Start the timer
start_time = time.time()
# umap
reducer_activations = umap.UMAP()
activations_umap_embedding = reducer_activations.fit_transform(activations_2d_scaled)

# End the timer
end_time = time.time()

# Calculate and print the time taken
elapsed_time = end_time - start_time
print(f"Time taken for umap: {elapsed_time:.4f} seconds")

ValueError: cannot assign slice from input of different size

In [135]:
# Start the timer
start_time = time.time()

# t-SNE
activations_tsne_embedding = TSNE(n_components=2).fit_transform(activations_2d_scaled)

# End the timer
end_time = time.time()

# Calculate and print the time taken
elapsed_time = end_time - start_time
print(f"Time taken for t-SNE: {elapsed_time:.4f} seconds")

Time taken for t-SNE: 995.7498 seconds


In [140]:
activations_tsne_embedding_reshaped = np.reshape(activations_tsne_embedding,(len(noise_levels)*sample_number,256,2))
activations_pca_embedding_reshaped = np.reshape(activations_pca_embedding,(len(noise_levels)*sample_number,256,2))

In [12]:
# activations_umap_embedding_reshaped = np.reshape(activations_umap_embedding,(len(noise_levels)*sample_number,256,2))

In [66]:
# noisy_df["bottleneck_tsne_embedding"] = activations_tsne_embedding_reshaped.tolist()
# noisy_df["bottleneck_pca_embedding"] = activations_pca_embedding_reshaped.tolist()
# # noisy_df["bottleneck_umap_embedding"] = activations_umap_embedding_reshaped.tolist()

# model_activations, ious_by_class, predictions
# overall_iou
noisy_df["overall_iou"] = overall_ious

# iou by class
iou_class_df = pd.DataFrame(ious_by_class,columns = ["other_iou","road_iou","sidewalk_iou","vegetation_iou","sky_iou","car_iou"])
df_full = pd.concat((noisy_df,iou_class_df),axis=1)

# paths of predictions
df_full["prediction_path"] = prediction_paths

In [67]:
df_full

,name,image_path,label_path,others,road,sidewalk,vegetation,sky,car,noise_level,...,umap_1,umap_2,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou,prediction_path
0,0.jpeg,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,0.002472,0,...,-0.698437,2.906299,0.823029,0.665488,0.949372,0.522178,0.559989,0.869651,0.066764,dataset\noise_data\predictions\0\0.jpeg
1,1.jpeg,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,0.000000,0,...,3.069255,1.605131,0.694122,0.598210,0.741562,0.246331,0.001455,0.480565,0.000000,dataset\noise_data\predictions\0\1.jpeg
2,2.jpeg,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,0.013748,0,...,2.172739,5.091578,0.904343,0.809380,0.970170,0.646972,0.827567,0.941647,0.194177,dataset\noise_data\predictions\0\2.jpeg
3,3.jpeg,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,0.010498,0,...,3.008074,4.119893,0.915451,0.850384,0.945627,0.707880,0.766526,0.884749,0.261182,dataset\noise_data\predictions\0\3.jpeg
4,4.jpeg,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,0.003937,0,...,-0.037946,3.517825,0.901062,0.769480,0.971295,0.837572,0.880520,0.296774,0.063342,dataset\noise_data\predictions\0\4.jpeg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,0.003189,5,...,0.165843,4.354430,0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841,dataset\noise_data\predictions\5\195.jpeg
1196,196.jpeg,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,0.002884,5,...,3.703897,2.663829,0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000,dataset\noise_data\predictions\5\196.jpeg
1197,197.jpeg,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,0.003357,5,...,3.205728,2.381734,0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000,dataset\noise_data\predictions\5\197.jpeg
1198,198.jpeg,dataset\noise_data\images\5\198.jpeg,dataset\cityscapes_labels_sample\198.jpeg,0.708481,0.000000,0.000000,0.236801,0.000381,0.054337,5,...,1.021540,3.467603,0.405075,0.382986,0.000000,0.000000,0.244767,0.000000,0.108215,dataset\noise_data\predictions\5\198.jpeg


In [68]:
df_full=df_full.rename(columns={"others": "other_ratio", "road": "road_ratio","sidewalk": "sidewalk_ratio","vegetation": "vegetation_ratio","sky": "sky_ratio","car": "car_ratio"})

In [69]:
df_full

,name,image_path,label_path,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,car_ratio,noise_level,...,umap_1,umap_2,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou,prediction_path
0,0.jpeg,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,0.002472,0,...,-0.698437,2.906299,0.823029,0.665488,0.949372,0.522178,0.559989,0.869651,0.066764,dataset\noise_data\predictions\0\0.jpeg
1,1.jpeg,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,0.000000,0,...,3.069255,1.605131,0.694122,0.598210,0.741562,0.246331,0.001455,0.480565,0.000000,dataset\noise_data\predictions\0\1.jpeg
2,2.jpeg,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,0.013748,0,...,2.172739,5.091578,0.904343,0.809380,0.970170,0.646972,0.827567,0.941647,0.194177,dataset\noise_data\predictions\0\2.jpeg
3,3.jpeg,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,0.010498,0,...,3.008074,4.119893,0.915451,0.850384,0.945627,0.707880,0.766526,0.884749,0.261182,dataset\noise_data\predictions\0\3.jpeg
4,4.jpeg,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,0.003937,0,...,-0.037946,3.517825,0.901062,0.769480,0.971295,0.837572,0.880520,0.296774,0.063342,dataset\noise_data\predictions\0\4.jpeg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,0.003189,5,...,0.165843,4.354430,0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841,dataset\noise_data\predictions\5\195.jpeg
1196,196.jpeg,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,0.002884,5,...,3.703897,2.663829,0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000,dataset\noise_data\predictions\5\196.jpeg
1197,197.jpeg,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,0.003357,5,...,3.205728,2.381734,0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000,dataset\noise_data\predictions\5\197.jpeg
1198,198.jpeg,dataset\noise_data\images\5\198.jpeg,dataset\cityscapes_labels_sample\198.jpeg,0.708481,0.000000,0.000000,0.236801,0.000381,0.054337,5,...,1.021540,3.467603,0.405075,0.382986,0.000000,0.000000,0.244767,0.000000,0.108215,dataset\noise_data\predictions\5\198.jpeg


In [70]:
# rearrange the columns in a fixed order
df_rearrange = df_full[['name','noise_level', 'image_path', 'label_path','prediction_path', 
                       'other_ratio', 'road_ratio', 'sidewalk_ratio','vegetation_ratio', 'sky_ratio', 'car_ratio', 
                       'simple_tsne_1','simple_tsne_2', 'meaningful_tsne_1', 'meaningful_tsne_2',"pca_1","pca_2",
                        "umap_1","umap_2",
        'overall_iou', 'other_iou',
       'road_iou', 'sidewalk_iou', 'vegetation_iou', 'sky_iou', 'car_iou']] # ,'bottleneck_umap_embedding'
#'bottleneck_tsne_embedding','bottleneck_pca_embedding',

In [71]:
df_rearrange

,name,noise_level,image_path,label_path,prediction_path,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,...,pca_2,umap_1,umap_2,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou
0,0.jpeg,0,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,dataset\noise_data\predictions\0\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,...,9617.183118,-0.698437,2.906299,0.823029,0.665488,0.949372,0.522178,0.559989,0.869651,0.066764
1,1.jpeg,0,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,dataset\noise_data\predictions\0\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,...,-373.437321,3.069255,1.605131,0.694122,0.598210,0.741562,0.246331,0.001455,0.480565,0.000000
2,2.jpeg,0,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,dataset\noise_data\predictions\0\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,...,8242.034722,2.172739,5.091578,0.904343,0.809380,0.970170,0.646972,0.827567,0.941647,0.194177
3,3.jpeg,0,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,dataset\noise_data\predictions\0\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,...,1137.464870,3.008074,4.119893,0.915451,0.850384,0.945627,0.707880,0.766526,0.884749,0.261182
4,4.jpeg,0,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,dataset\noise_data\predictions\0\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,...,12602.637921,-0.037946,3.517825,0.901062,0.769480,0.971295,0.837572,0.880520,0.296774,0.063342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,5,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,dataset\noise_data\predictions\5\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,...,-1282.660143,0.165843,4.354430,0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841
1196,196.jpeg,5,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,dataset\noise_data\predictions\5\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,...,-9698.227061,3.703897,2.663829,0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000
1197,197.jpeg,5,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,dataset\noise_data\predictions\5\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,...,-9677.253907,3.205728,2.381734,0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000
1198,198.jpeg,5,dataset\noise_data\images\5\198.jpeg,dataset\cityscapes_labels_sample\198.jpeg,dataset\noise_data\predictions\5\198.jpeg,0.708481,0.000000,0.000000,0.236801,0.000381,...,-1289.474219,1.021540,3.467603,0.405075,0.382986,0.000000,0.000000,0.244767,0.000000,0.108215


In [72]:
print(len(df_rearrange.columns))

26


In [73]:
df_rearrange.to_csv("noisy_data_sample_200.csv",index=True)

TODO: add the bottleneck activatiosn embedding from kaggle output

and then also don't forget to add the "id"

# Generate umap embedding with the saved activation

In [2]:
pickle_file = "D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\jupyter\\noise_activations_scaled_200_samples_9.22.pkl"
with open(pickle_file, 'rb') as f:
    activations = pickle.load(f)

In [19]:
df_full = pd.read_csv("D:\zsh\graduation\Graduation-project-domain-shift-image-2-image\jupyter\\noisy_data_sample_200.csv",index_col="id")

In [20]:
df_full

,name,noise_level,image_path,label_path,prediction_path,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,...,pca_2,umap_1,umap_2,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou
id,,,,,,,,,,,,,,,,,,,,,
0,0.jpeg,0,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,dataset\noise_data\predictions\0\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,...,9617.183118,-0.698437,2.906299,0.823029,0.665488,0.949372,0.522178,0.559989,0.869651,0.066764
1,1.jpeg,0,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,dataset\noise_data\predictions\0\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,...,-373.437321,3.069255,1.605131,0.694122,0.598210,0.741562,0.246331,0.001455,0.480565,0.000000
2,2.jpeg,0,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,dataset\noise_data\predictions\0\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,...,8242.034722,2.172739,5.091578,0.904343,0.809380,0.970170,0.646972,0.827567,0.941647,0.194177
3,3.jpeg,0,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,dataset\noise_data\predictions\0\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,...,1137.464870,3.008074,4.119893,0.915451,0.850384,0.945627,0.707880,0.766526,0.884749,0.261182
4,4.jpeg,0,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,dataset\noise_data\predictions\0\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,...,12602.637920,-0.037946,3.517825,0.901062,0.769480,0.971295,0.837572,0.880520,0.296774,0.063342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,5,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,dataset\noise_data\predictions\5\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,...,-1282.660143,0.165843,4.354430,0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841
1196,196.jpeg,5,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,dataset\noise_data\predictions\5\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,...,-9698.227061,3.703897,2.663829,0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000
1197,197.jpeg,5,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,dataset\noise_data\predictions\5\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,...,-9677.253907,3.205728,2.381734,0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000


In [3]:
activations.shape # scaled and ready to do umap on it

(307200, 1024)

In [4]:
print(np.isnan(activations).sum())
print(np.isinf(activations).sum())

0
0


In [5]:
print(activations.dtype)

float32


In [21]:
# Start the timer
start_time = time.time()
activations_pca_embedding = PCA(n_components=2).fit_transform(activations)

# End the timer
end_time = time.time()

# Calculate and print the time taken
elapsed_time = end_time - start_time
print(f"Time taken for PCA: {elapsed_time:.4f} seconds")

Time taken for PCA: 26.0737 seconds


In [22]:
sample_number = 200
noise_levels = [0,1,2,3,4,5]

In [23]:
activations_pca_embedding_reshaped = np.reshape(activations_pca_embedding,(len(noise_levels)*sample_number,256,2))
df_full["bottleneck_pca_embedding"] = activations_pca_embedding_reshaped.tolist()

In [18]:
# Start the timer
start_time = time.time()

# t-SNE
activations_tsne_embedding = TSNE(n_components=2).fit_transform(activations)

# End the timer
end_time = time.time()

# Calculate and print the time taken
elapsed_time = end_time - start_time
print(f"Time taken for t-SNE: {elapsed_time:.4f} seconds")

Time taken for t-SNE: 139628.2854 seconds


In [24]:
activations_tsne_embedding_reshaped = np.reshape(activations_tsne_embedding,(len(noise_levels)*sample_number,256,2))
df_full["bottleneck_tsne_embedding"] = activations_tsne_embedding_reshaped.tolist()

In [25]:
df_full

,name,noise_level,image_path,label_path,prediction_path,other_ratio,road_ratio,sidewalk_ratio,vegetation_ratio,sky_ratio,...,umap_2,overall_iou,other_iou,road_iou,sidewalk_iou,vegetation_iou,sky_iou,car_iou,bottleneck_pca_embedding,bottleneck_tsne_embedding
id,,,,,,,,,,,,,,,,,,,,,
0,0.jpeg,0,dataset\noise_data\images\0\0.jpeg,dataset\cityscapes_labels_sample\0.jpeg,dataset\noise_data\predictions\0\0.jpeg,0.372009,0.320770,0.100510,0.183990,0.020248,...,2.906299,0.823029,0.665488,0.949372,0.522178,0.559989,0.869651,0.066764,"[[-0.013326136395335197, -0.30401840806007385]...","[[-8.655488967895508, 15.489809036254883], [-1..."
1,1.jpeg,0,dataset\noise_data\images\0\1.jpeg,dataset\cityscapes_labels_sample\1.jpeg,dataset\noise_data\predictions\0\1.jpeg,0.539703,0.414398,0.041168,0.000565,0.004166,...,1.605131,0.694122,0.598210,0.741562,0.246331,0.001455,0.480565,0.000000,"[[0.1603696197271347, -0.22683094441890717], [...","[[-14.934941291809082, 11.525232315063477], [-..."
2,2.jpeg,0,dataset\noise_data\images\0\2.jpeg,dataset\cityscapes_labels_sample\2.jpeg,dataset\noise_data\predictions\0\2.jpeg,0.462753,0.354187,0.029633,0.121323,0.018356,...,5.091578,0.904343,0.809380,0.970170,0.646972,0.827567,0.941647,0.194177,"[[0.019657980650663376, -0.3139321804046631], ...","[[-1.8404988050460815, 13.750861167907715], [-..."
3,3.jpeg,0,dataset\noise_data\images\0\3.jpeg,dataset\cityscapes_labels_sample\3.jpeg,dataset\noise_data\predictions\0\3.jpeg,0.425674,0.381119,0.065292,0.101120,0.016296,...,4.119893,0.915451,0.850384,0.945627,0.707880,0.766526,0.884749,0.261182,"[[0.1766330897808075, -0.19020713865756989], [...","[[-16.590892791748047, 5.171955108642578], [-0..."
4,4.jpeg,0,dataset\noise_data\images\0\4.jpeg,dataset\cityscapes_labels_sample\4.jpeg,dataset\noise_data\predictions\0\4.jpeg,0.400284,0.329453,0.090469,0.174393,0.001465,...,3.517825,0.901062,0.769480,0.971295,0.837572,0.880520,0.296774,0.063342,"[[0.924606204032898, 0.10404495894908905], [1....","[[-13.471735000610352, 9.674683570861816], [-1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,195.jpeg,5,dataset\noise_data\images\5\195.jpeg,dataset\cityscapes_labels_sample\195.jpeg,dataset\noise_data\predictions\5\195.jpeg,0.409088,0.489960,0.004944,0.091309,0.001511,...,4.354430,0.524734,0.418835,0.417183,0.013165,0.221730,0.017021,0.030841,"[[0.520260214805603, -0.023931534960865974], [...","[[-12.483431816101074, -0.7725005149841309], [..."
1196,196.jpeg,5,dataset\noise_data\images\5\196.jpeg,dataset\cityscapes_labels_sample\196.jpeg,dataset\noise_data\predictions\5\196.jpeg,0.167236,0.348999,0.140472,0.178207,0.162201,...,2.663829,0.298218,0.155384,0.268548,0.114600,0.115912,0.189091,0.000000,"[[0.3765714466571808, -0.041820090264081955], ...","[[-4.8248372077941895, -16.330381393432617], [..."
1197,197.jpeg,5,dataset\noise_data\images\5\197.jpeg,dataset\cityscapes_labels_sample\197.jpeg,dataset\noise_data\predictions\5\197.jpeg,0.517563,0.249649,0.147522,0.074539,0.007370,...,2.381734,0.500778,0.416154,0.506934,0.075644,0.124599,0.083700,0.000000,"[[0.10845088958740234, 0.3767417371273041], [0...","[[-17.50540542602539, 6.707708358764648], [-7...."


In [14]:
# # example data to check if umap is working as intended
# test_data = np.random.rand(1000, 1024)
# test_embedding = umap.UMAP().fit_transform(test_data)

In [27]:
# rearrange the columns in a fixed order
df_rearrange = df_full[['name','noise_level', 'image_path', 'label_path','prediction_path', 
                       'other_ratio', 'road_ratio', 'sidewalk_ratio','vegetation_ratio', 'sky_ratio', 'car_ratio', 
                       'simple_tsne_1','simple_tsne_2', 'meaningful_tsne_1', 'meaningful_tsne_2',"pca_1","pca_2",
                        "umap_1","umap_2",'bottleneck_pca_embedding','bottleneck_tsne_embedding',
        'overall_iou', 'other_iou',
       'road_iou', 'sidewalk_iou', 'vegetation_iou', 'sky_iou', 'car_iou']] # ,'bottleneck_pca_embedding',

In [29]:
df_rearrange.columns

Index(['name', 'noise_level', 'image_path', 'label_path', 'prediction_path',
       'other_ratio', 'road_ratio', 'sidewalk_ratio', 'vegetation_ratio',
       'sky_ratio', 'car_ratio', 'simple_tsne_1', 'simple_tsne_2',
       'meaningful_tsne_1', 'meaningful_tsne_2', 'pca_1', 'pca_2', 'umap_1',
       'umap_2', 'bottleneck_pca_embedding', 'bottleneck_tsne_embedding',
       'overall_iou', 'other_iou', 'road_iou', 'sidewalk_iou',
       'vegetation_iou', 'sky_iou', 'car_iou'],
      dtype='object')

In [30]:
df_rearrange.to_csv("noisy_data_sample_200.csv",index=True)